In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

#dotenv_path = Path(__file__).resolve().parent / '.env'  #Não funciona em notebook
load_dotenv("../.env")
api_key = os.getenv("ATF-image")

In [2]:
import re
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd
import io

In [3]:
def link_drive_direto(link: str) -> str:
    """
    Recebe um link de compartilhamento do Google Drive e retorna o link direto para a imagem.
    """
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', link)
    if match:
        file_id = match.group(1)
        return f"https://drive.google.com/uc?export=view&id={file_id}"
    else:
        return link  # Se não for do Google Drive, retorna o link original

In [4]:
page1_link = "https://drive.google.com/file/d/1tO4VIdkF8gcVRmJQ-7-pzECJycoMQsd_/view?usp=sharing"
page2_link = "https://drive.google.com/file/d/1ASZjgPIjmjVtPtVNpsj3wOO_KXpb6E3_/view?usp=sharing" #140KB  #Deu 2541 tokens entrada e 98 tokens saida

page1 = link_drive_direto(page1_link)
page2 = link_drive_direto(page2_link)

In [5]:
llm = ChatOpenAI(
    api_key=api_key,
    model="gpt-4.1-mini-2025-04-14", #gpt-4.1-mini-2025-04-14         #gpt-4.1-2025-04-14      #gpt-4o-2024-08-06   #o4-mini-2025-04-16
   # temperature=0,
)


In [ ]:
prompt = """
Você receberá imagens de um extrato de uma conta bancária, onde os débitos podem ser representados pela cor vermelha, por um sinal de menos, pela letra 'D', ou por algum outro sinal, enquanto que os créditos podem ser representados pela cor azul, por um sinal de +, pela letra 'C', ou por algum outro sinal.
Geralmente os dados estão estruturados, e seguem a forma de uma tabela. Entenda qual coluna tem as informações desajadas. Por exemplo: Pode receber uma imagem que tem uma coluna com nome valor e outra com nome saldo, você não deve considerar os valores de saldo nas sua operações. 
Retorne um JSON com uma lista dos valores dos débitos e outra com os valores dos créditos das movimentações financeiras desta conta. Desconsidere os dados de saldo bancário, considerando apenas as movimentações.
1.Retorne um o JSON puro, sem explicações, no seguinte formato:
{
    "debito": [...],
    "credito": [...]
}

2. Retorne um segundo json contendo as informações da origem das transferências. Exemplo:
Se a Maria Duarte recebeu 3 transferências pix nos valores de: 120, 110 e 20. O somatório deve estar no segundo Json:
{
    Transferência Pix, Maria Duarte (Crédito): 250
}
"""

In [6]:
prompt = """
Você receberá imagens de um extrato de uma conta bancária, onde os débitos podem ser representados pela cor vermelha, por um sinal de menos, pela letra 'D', ou por algum outro sinal, enquanto que os créditos podem ser representados pela cor azul, por um sinal de +, pela letra 'C', ou por algum outro sinal.
Geralmente os dados estão estruturados, e seguem a forma de uma tabela. Entenda qual coluna tem as informações desajadas. Por exemplo: Pode receber uma imagem que tem uma coluna com nome valor e outra com nome saldo, você não deve considerar os valores de saldo nas suas operações. 
Os valores dos débitos e outra com os valores dos créditos das movimentações financeiras desta conta. Desconsidere os dados de saldo bancário, considerando apenas as movimentações.
1. O formato deve ser csv do tipo:    

tipo,valor,origem, data
debito,22.97,PIX QRS IFOOD.COM A27/03,28/03/2025
debito,25.11,PIX QRS SHPP BRASIL26/03,04/05/2025
credito,12.19,DEV PIX SHPP BRASIL28/03,05/05/2025

Caso não encontre algum campo deixe vazio;
*IMPORTANTE*: Considere apenas as movimentações financeiras. Não inclua dados de saldo bancário. 
Em alguns tipo de extrato, pode ser que tenha o "SALDO DO DIA" na coluna de movimentações, você NÃO DEVE colocar essa linha no csv.

"""

In [7]:
messages = [
    SystemMessage(content= prompt),
    HumanMessage(
        content=[{"type": "image_url", "image_url": {"url": page1}}        ]
    )
]

In [8]:
response = llm.invoke(messages)

In [9]:
print(response)

content='tipo,valor,origem,data\ndebito,663.80,CHEQUE AVULSO DOC.: 000.016,03/10/2025\ncredito,11000.00,CRÉD.TRANSF.POU.INT 4390 - 627130399 ÉRICO FANTINEL DE BORTOLI DOC.: 9645166,14/10/2025\ndebito,9.68,PIX EMIT.OUTRA IF Pagamento Pix 79.863.890 0004-68 DOC.: Pix,14/10/2025\ncredito,9.68,EST.PIX EMIT.OUT.IF Estorno Pix 79.863.890/0004-68 DOC.: Pix,14/10/2025\ndebito,5000.00,PIX EMIT.OUTRA IF Pagamento Pix ***.071.779-** DOC.: Pix,14/10/2025\ndebito,543.00,PIX EMIT.OUTRA IF Pagamento Pix 42.496.258 0001-70 DOC.: Pix,14/10/2025\ndebito,110.00,PIX EMIT.OUTRA IF Pagamento Pix 14.029.801 0001-94 DOC.: Pix,14/10/2025\ndebito,366.00,PIX EMIT.OUTRA IF Pagamento Pix ***.718.259-** DOC.: Pix,14/10/2025\ndebito,920.00,PIX EMIT.OUTRA IF Pagamento Pix ***.071.779-** DOC.: Pix,14/10/2025\ndebito,55.90,DEB PACOTE SERVIÇOS DOC.: 129,14/10/2025\ndebito,980.00,PIX EMIT.OUTRA IF Pagamento Pix 42.496.258 0001-70 DOC.: Pix,15/10/2025\ndebito,27.50,PIX EMIT.OUTRA IF Pagamento Pix,15/10/2025' additional_kw

In [11]:
response.usage_metadata['output_tokens']

427

In [48]:
extrato_csv = response.content
df = pd.read_csv(io.StringIO(extrato_csv))

In [61]:
df['origem'].unique()

array(['CHEQUE AVULSO DOC.: 000.016',
       'CRÉD.TRANSF.POU.INT 4390 - 627130399 ÉRICO FANTINEL DE BORTOLI DOC.: 9645166',
       'PIX EMIT.OUTRA IF Pagamento Pix 79.863.890 0004-68 DOC.: Pix',
       'EST.PIX EMIT.OUT.IF Estorno Pix 79.863.890/0004-68 DOC.: Pix',
       'PIX EMIT.OUTRA IF Pagamento Pix ***.071.779-** DOC.: Pix',
       'PIX EMIT.OUTRA IF Pagamento Pix 42.496.258 0001-70 DOC.: Pix',
       'PIX EMIT.OUTRA IF Pagamento Pix 14.029.801 0001-94 DOC.: Pix',
       'PIX EMIT.OUTRA IF Pagamento Pix ***.718.259-** DOC.: Pix',
       'DEB PACOTE SERVIÇOS DOC.: 129', 'PIX EMIT.OUTRA IF Pagamento Pix'],
      dtype=object)

In [ ]:
tota_credito = df[df['tipo'] == 'credito']['valor'].sum()
total_debito = df[df['tipo'] == 'debito']['valor'].sum()
total_liquido = tota_credito - total_debito